### Compute Summary Statistics for merged salary dataset UC, Michigan, Texas and Illinois

update the uc_salary_summary.xlsx Excel file so that it has summary statistics for all universities (i.e., add Michigan, Texas, and Illinois). Each tab within the spreadsheet could have the fields in the rows (as it currently has), and then just add the other universities as columns (e.g., Berkeley, UCLA, ..., Merced, UC All, Ann Arbor, Dearborn, Flint, Michigan All, etc). I'm mainly interested in these tabs:

1. Number of salary observations
2. Mean salary (pick regular or gross, whichever is more consistent across schools)
3. SD of salary
4. Percentage increase in salary at tenure (similar to your current 'Tenure' tab, but just use the mean and put the universities on the columns)
5. Correlation coefficient between salary in year t and total citations by year t


In [1]:
import os
import warnings
warnings.simplefilter('ignore')

import pandas as pd
#pd.options.display.float_format = "{:,.2f}".format

import numpy as np
from matplotlib import pyplot as plt
#import seaborn as sns
%matplotlib inline
pd.options.display.float_format = "{:,.2f}".format

os.chdir('/Users/apple/Desktop/research_fellow_documents/merge_sum_salary/')

In [2]:
df = pd.read_csv('salary_data/salary_all.csv')

In [3]:
len(df)

664921

## 1. Number of salary observations
group by university and field, count unique UIDs, since one person could hold multiple positions ('professor' & 'dean') for example within a department.

In [4]:
m1 = df.groupby(['university','field'])['uid'].nunique().reset_index()

m1 = m1.pivot(index="field", columns="university", values="uid").reset_index()
m1.loc[-1] = m1[m1.columns[1:]].sum(axis = 0)
m1.index = m1.index + 1  # shifting index
m1 = m1.sort_index()
m1['field'][0] = 'All'

m1['schools with field'] = m1.apply(lambda x: x.notnull().sum(), axis='columns')
m1 = m1.sort_values(['schools with field'], ascending=False)
m1.reset_index(drop=True, inplace=True)

#save file
m1.to_excel('salary_summary/salary_summary.xlsx', index = False)

#visualize
m1.head()

university,field,Ann Arbor,Arlington,Austin,Berkeley,Chicago,Dallas,Davis,Dearborn,El Paso,...,Riverside,San Antonio,San Diego,Santa Barbara,Santa Cruz,Springfield,System Offices,Tyler,Urbana-Champaign,schools with field
0,All,"14,623.00",507.00,"1,839.00","1,483.00","8,935.00",465.00,"2,327.00",659.00,748.00,...,783.00,201.00,"2,178.00","1,257.00",423.00,441.00,70.00,172.00,"7,220.00",25
1,History,188.00,12.00,26.00,42.00,50.00,4.00,35.00,NaN,18.00,...,30.00,11.00,32.00,60.00,19.00,15.00,NaN,3.00,75.00,22
2,Economics,167.00,7.00,40.00,36.00,33.00,11.00,34.00,NaN,18.00,...,18.00,1.00,34.00,29.00,21.00,2.00,NaN,NaN,82.00,21
3,Mechanical Engineering,127.00,27.00,48.00,36.00,125.00,18.00,27.00,49.00,15.00,...,21.00,11.00,38.00,31.00,NaN,NaN,NaN,5.00,160.00,21
4,Physics and Astronomy,149.00,25.00,52.00,54.00,102.00,16.00,46.00,NaN,13.00,...,39.00,10.00,53.00,45.00,29.00,NaN,NaN,NaN,170.00,21


## 2. Mean salary (gross salary)

convert to year-uid-field panel and use gross_pay_sum column. <br>
Note: same individual with multiple affiliated departments are counted in different departments. 

In [5]:
m2 = df.drop_duplicates(['uid','yr','field']).groupby(['university','field'])['gross_pay_sum'].mean().reset_index()
m2 = m2.pivot(index="field", columns="university", values="gross_pay_sum").reset_index()
m2.loc[-1] = m2[m2.columns[1:]].mean(axis = 0)
m2.index = m2.index + 1  # shifting index
m2 = m2.sort_index()
m2['field'][0] = 'All'

m2['schools with field'] = m2.apply(lambda x: x.notnull().sum(), axis='columns')
m2 = m2.sort_values(['schools with field'], ascending=False)
m2.reset_index(drop=True, inplace=True)

#save file
m2.to_excel('salary_summary/mean_gross_pay.xlsx', index = False)

#visualize
m2.head()

university,field,Ann Arbor,Arlington,Austin,Berkeley,Chicago,Dallas,Davis,Dearborn,El Paso,...,Riverside,San Antonio,San Diego,Santa Barbara,Santa Cruz,Springfield,System Offices,Tyler,Urbana-Champaign,schools with field
0,All,"151,453.98","97,015.34","171,876.24","161,909.06","116,677.51","102,183.18","173,708.66","103,111.10","90,964.94",...,"118,984.24","86,953.35","187,009.58","161,402.24","123,606.75","72,302.08","112,155.56","99,789.37","113,524.98",25
1,History,"119,340.06","80,292.67","153,325.93","132,281.72","108,460.65","53,263.20","122,441.26",NaN,"70,098.73",...,"110,164.89","71,057.38","102,803.57","128,181.21","103,750.76","92,425.00",NaN,"70,869.80","118,660.54",22
2,Economics,"208,016.61","98,728.97","226,184.45","299,050.99","138,705.08","118,570.30","150,109.82",NaN,"134,163.53",...,"143,929.41","80,571.50","198,515.80","215,901.60","164,401.33","72,563.43",NaN,NaN,"153,833.44",21
3,Mechanical Engineering,"139,111.04","113,449.77","186,331.43","190,568.93","108,596.86","106,368.01","153,455.43","113,123.71","113,000.43",...,"147,246.23","79,563.65","174,373.47","197,240.55",NaN,NaN,NaN,"94,205.78","136,541.39",21
4,Physics and Astronomy,"110,885.35","86,166.00","163,708.05","190,449.19","105,951.33","101,100.90","151,411.62",NaN,"77,830.62",...,"142,902.02","88,018.10","173,552.22","188,797.70","163,621.45",NaN,NaN,NaN,"133,484.25",21


## 3. SD salary (gross salary)

convert to year-uid-field panel and use gross_pay_sum column. <br>
Note: same individual with multiple affiliated departments are counted in different departments. 

In [6]:
m2 = df.drop_duplicates(['uid','yr','field']).groupby(['university','field'])['gross_pay_sum'].std().reset_index()
m2 = m2.pivot(index="field", columns="university", values="gross_pay_sum").reset_index()
m2.loc[-1] = m2[m2.columns[1:]].mean(axis = 0)
m2.index = m2.index + 1  # shifting index
m2 = m2.sort_index()
m2['field'][0] = 'All'

m2['schools with field'] = m2.apply(lambda x: x.notnull().sum(), axis='columns')
m2 = m2.sort_values(['schools with field'], ascending=False)
m2.reset_index(drop=True, inplace=True)

#save file
m2.to_excel('salary_summary/sd_gross_pay.xlsx', index = False)

#visualize
m2.head()

university,field,Ann Arbor,Arlington,Austin,Berkeley,Chicago,Dallas,Davis,Dearborn,El Paso,...,Riverside,San Antonio,San Diego,Santa Barbara,Santa Cruz,Springfield,System Offices,Tyler,Urbana-Champaign,schools with field
0,All,"68,158.41","54,599.24","69,767.93","64,368.15","64,995.53","50,755.80","71,220.02","37,461.11","45,460.53",...,"52,471.37","46,221.77","87,956.53","61,062.25","41,140.86","20,338.62","74,995.91","26,341.71","58,793.50",25
1,History,"76,353.89","50,578.01","78,571.54","44,411.87","49,224.43","34,083.34","44,308.33",NaN,"41,523.41",...,"39,177.72","34,248.06","43,379.17","53,178.13","40,659.39","63,111.32",NaN,"8,384.87","80,771.42",22
2,Physics and Astronomy,"55,687.38","55,526.87","101,073.26","64,825.16","86,483.25","46,208.34","47,554.84",NaN,"34,083.39",...,"54,384.27","44,155.38","61,405.70","66,364.71","61,341.08",NaN,NaN,NaN,"81,511.65",21
3,Mechanical Engineering,"56,595.33","65,739.27","74,789.02","61,472.28","52,053.12","49,462.09","56,571.56","52,919.59","89,506.76",...,"60,172.15","49,153.58","74,860.84","89,108.58",NaN,NaN,NaN,"21,627.85","57,433.94",21
4,Economics,"143,748.45","39,473.47","119,712.19","110,499.78","60,869.89","61,347.78","59,466.93",NaN,"86,165.61",...,"49,604.46","28,484.78","87,150.11","92,646.60","52,902.35","3,345.09",NaN,NaN,"72,567.88",21


## 4. 90-10 percentile salary (gross salary)

convert to year-uid-field panel and use gross_pay_sum column. <br>
Note: same individual with multiple affiliated departments are counted in different departments. 

In [7]:
#10-90 percentile salary difference
m3 = df.drop_duplicates(['uid','yr','field']).groupby(['university','field'])['gross_pay_sum'].quantile(q = 0.9) - df.drop_duplicates(['uid','yr','field']).groupby(['university','field'])['gross_pay_sum'].quantile(q = 0.1) 
m3 = m3.unstack().T.reset_index()
gp_mt = df.drop_duplicates(['uid','yr','field']).groupby(['university'])['gross_pay_sum'].quantile(q = 0.9) - df.drop_duplicates(['uid','yr','field']).groupby(['university'])['gross_pay_sum'].quantile(q = 0.1) 
m3.loc[-1] = gp_mt
m3.index = m3.index + 1  # shifting index
m3 = m3.sort_index()
m3['field'][0] = 'All'
m3['schools with field'] = m3.apply(lambda x: x.notnull().sum(), axis='columns')

m3 = m3.sort_values(['schools with field'], ascending=False)
m3.reset_index(drop=True, inplace=True)

#save file
m3.to_excel('salary_summary/90_10_gap_gross_pay.xlsx', index = False)

#visualize
m3.head()

university,field,Ann Arbor,Arlington,Austin,Berkeley,Chicago,Dallas,Davis,Dearborn,El Paso,...,Riverside,San Antonio,San Diego,Santa Barbara,Santa Cruz,Springfield,System Offices,Tyler,Urbana-Champaign,schools with field
0,All,"169,400.00","73,939.40","125,867.20","243,706.00","167,192.83","91,009.80","242,583.47","101,131.43","73,173.00",...,"175,267.36","64,409.00","309,046.94","170,479.00","131,046.34","71,749.00","99,445.84","69,933.70","125,179.00",25
1,History,"131,616.00","114,812.70","208,662.20","116,241.90","101,265.10","82,778.60","119,673.66",NaN,"99,713.00",...,"96,066.00","84,963.10","102,657.20","137,813.10","69,735.50","120,184.00",NaN,"13,671.20","91,535.88",22
2,Economics,"342,082.00","112,257.90","266,453.90","274,846.90","145,419.04","157,761.20","163,734.09",NaN,"231,088.20",...,"129,008.50","51,323.20","221,289.00","225,064.00","129,316.40","6,070.60",NaN,NaN,"186,008.00",21
3,Mechanical Engineering,"118,178.00","174,764.00","177,048.40","148,129.90","123,393.37","113,233.80","130,140.40","116,928.70","250,078.90",...,"135,636.60","134,484.10","165,283.14","219,963.50",NaN,NaN,NaN,"45,336.20","136,723.52",21
4,Physics and Astronomy,"88,042.00","141,499.20","206,793.40","150,255.18","98,635.00","101,435.40","118,754.78",NaN,"84,531.80",...,"128,713.60","95,132.40","143,421.30","139,211.64","152,286.20",NaN,NaN,NaN,"120,590.00",21


## 5. Percentage increase in salary at tenure 
(similar to your current 'Tenure' tab, but just use the mean and put the universities on the columns)


In [8]:
df2 = df.drop_duplicates(['uid','yr']).sort_values(['uid','yr'])

In [9]:
def find_tenure_yr(df):
    """
    Include three extra columns to df for previous years rank and salary
    """
    df['prev_rank'] = df['rank'].shift(1)
    df['prev_gross_pay'] = df['gross_pay_sum'].shift(1)
    #df['prev_regular_pay'] = df['regular_pay'].shift(1)
    return df
df3 = df2.groupby(['uid']).apply(find_tenure_yr)

In [10]:
def tenure_sum_stats(df, prev, now, limit_field = True):
    """
    prev - rank in previous year - ['Assistant','Associate']
    now - rank in previous year - ['Associate','Full']
    """
    fields = ['Economics','Mathematics','Sociology',
              'Computer Science', 'Business', 'Internal Medicine', 
              'Psychology','Chemistry','Cellular Biology',
              'Foreign Language/Area Studies','Physics and Astronomy','Medicine','Law']
    
    if limit_field == True:
        df = df[df.field.isin(fields)]
    else:
        df = df
    tenure = df[(df['prev_rank'] == prev) & (df['rank'] == now)]
    
    #deal with case of division by zero - remove those rows
    tenure = tenure[tenure['prev_gross_pay']!=0]
    #tenure = tenure[tenure['prev_regular_pay']!=0]
    
    tenure['dif_gross'] = tenure['gross_pay'] - tenure['prev_gross_pay']
    tenure['dif_gross_pct'] = tenure['dif_gross'] / tenure['prev_gross_pay']
    #tenure['dif_regular'] = tenure['regular_pay'] - tenure['prev_regular_pay'] 
    #tenure['dif_regular_pct'] = tenure['dif_regular'] / tenure['prev_regular_pay'] * 100
    
    #t1 = tenure.groupby(['field'])['dif_gross_pct'].agg(['mean','median','count','max','min']).sort_values(['median'], ascending=False).add_suffix('_gross').reset_index()
    t1 = tenure.groupby(['field'])['dif_gross_pct'].agg(['mean','count']).sort_values(['mean'], ascending=False).add_suffix('_gross').reset_index()
    #t2 = tenure.groupby(['field'])['dif_regular_pct'].agg(['mean','median','count','max','min']).sort_values(['median'], ascending=False).add_suffix('_regular').reset_index()
    #return t1.merge(t2, on=['field'])
    return t1

In [11]:
univ_list = set(df3['university'])
data_list = []
for item in univ_list:
    df = df3[df3['university'] == item]
    data = tenure_sum_stats(df,'Assistant','Associate', limit_field = False)
    data['university'] = item
    print('university {} done.'.format(item))
    data_list.append(data)

university Berkeley done.
university El Paso done.
university Davis done.
university San Antonio done.
university San Francisco done.
university Hastings College Of Law done.
university Austin done.
university Flint done.
university Dearborn done.
university Dallas done.
university Springfield done.
university Merced done.
university Tyler done.
university Arlington done.
university Chicago done.
university Los Angeles done.
university Irvine done.
university DANR done.
university Ann Arbor done.
university San Diego done.
university UCOP done.
university Santa Barbara done.
university Permian Basin done.
university Urbana-Champaign done.
university Santa Cruz done.
university Rio Grande Valley done.
university Riverside done.
university System Offices done.


In [12]:
result = pd.concat(data_list, ignore_index=True)
result2 = result.drop(columns=['count_gross'])
result2 = result2.pivot(index="field", columns="university", values="mean_gross").reset_index()
result2.loc[-1] = result2[result2.columns[1:]].mean(axis = 0)
result2.index = result2.index + 1  # shifting index
result2 = result2.sort_index()
result2['field'][0] = 'All'

result2['schools with field'] = result2.apply(lambda x: x.notnull().sum(), axis='columns')
result2 = result2.sort_values(['schools with field'], ascending=False)
result2.reset_index(drop=True, inplace=True)

In [13]:
result2.to_excel('salary_summary/tenure.xlsx', index = False)

In [14]:
univ_list = set(df3['university'])
data_list2 = []
for item in univ_list:
    df = df3[df3['university'] == item]
    data = tenure_sum_stats(df,'Associate','Full', limit_field = False)
    data['university'] = item
    print('university {} done.'.format(item))
    data_list2.append(data)

university Berkeley done.
university El Paso done.
university Davis done.
university San Antonio done.
university San Francisco done.
university Hastings College Of Law done.
university Austin done.
university Flint done.
university Dearborn done.
university Dallas done.
university Springfield done.
university Merced done.
university Tyler done.
university Arlington done.
university Chicago done.
university Los Angeles done.
university Irvine done.
university DANR done.
university Ann Arbor done.
university San Diego done.
university UCOP done.
university Santa Barbara done.
university Permian Basin done.
university Urbana-Champaign done.
university Santa Cruz done.
university Rio Grande Valley done.
university Riverside done.
university System Offices done.


In [15]:
full = pd.concat(data_list2, ignore_index=True)
result2 = full.drop(columns=['count_gross'])
result2 = result2.pivot(index="field", columns="university", values="mean_gross").reset_index()
result2.loc[-1] = result2[result2.columns[1:]].mean(axis = 0)
result2.index = result2.index + 1  # shifting index
result2 = result2.sort_index()
result2['field'][0] = 'All'

result2['schools with field'] = result2.apply(lambda x: x.notnull().sum(), axis='columns')
result2 = result2.sort_values(['schools with field'], ascending=False)
result2.reset_index(drop=True, inplace=True)
result2.to_excel('salary_summary/assoc_full.xlsx', index = False)

## 5. Correlation coefficient between salary in year t and quality-weighted publications by year t


In [16]:
#load uc publication and salary data
#load uc publication and salary data
uc_pub = pd.read_stata('/Users/apple/Dropbox/web_scrapping_UC/temp/scopus.dta')
texas_pub = pd.read_csv('/Users/apple/Dropbox/web_scrapping_UC/temp/scopus_texas.csv')
illi_pub = pd.read_csv('/Users/apple/Dropbox/web_scrapping_UC/temp/scopus_illinois.csv')
mich_pub = pd.read_csv('/Users/apple/Dropbox/web_scrapping_UC/temp/scopus_michigan.csv')
#load citescore for journals
journal_rnk = pd.read_csv('/Users/apple/Dropbox/web_scrapping_UC/scopus/scopus_aux/citescore.csv')
journal_rnk.rename(columns = {'year':'pub_yr'}, inplace = True)
journal_rnk2019 = pd.read_csv('/Users/apple/Desktop/research_fellow_documents/data_analysis2021107/scopus_journal_ranking_2019.csv')

#load research area code
asjc = pd.read_csv('/Users/apple/Dropbox/web_scrapping_UC/scopus/scopus_aux/scopus_journal_classify_updated_copy.csv')


In [17]:
all_pub = pd.concat([uc_pub, texas_pub, illi_pub, mich_pub], ignore_index = False)

In [18]:
#merge citescore into publication data
### articles with no citescore is filled with zero ... including all publications before 1990 ###
all_pub = all_pub.merge(journal_rnk[['source_id','pub_yr','citescore']], on = ['pub_yr','source_id'], how = 'left')
all_pub['citescore'] = all_pub['citescore'].fillna(0)

#standardize columns and make consistent with salary data
all_pub['first_name'] = all_pub['first_name'].str.upper()
all_pub['last_name'] = all_pub['last_name'].str.upper()

all_pub.rename(columns = {'pub_yr':'yr'}, inplace = True)

In [19]:
all_pub.columns

Index(['last_name', 'first_name', 'university', 'author_id', 'scopus_id',
       'affil_id', 'pub_type', 'pub_date', 'yr', 'cites', 'num_coauthor',
       'rank_author', 'source_id', 'source_type', 'affil_name', 'affil_city',
       'affil_country', 'citescore'],
      dtype='object')

In [20]:
df.columns

Index(['university', 'uid', 'yr', 'first_name', 'last_name', 'department',
       'field', 'field_macro', 'field_micro', 'title', 'rank', 'gross_pay',
       'regular_pay', 'overtime_pay', 'other_pay', 'suspicious',
       'gross_pay_sum', 'regular_pay_sum', 'name', 'univ_sys', 'prof_type',
       'fte', 'pay_term', 'gen_fund', 'college', 'division', 'gen_fund_sum',
       'college_code', 'department_code', 'emp_no', 'multiple_jobs',
       'fte_total', 'tenure', 'emplclass', 'cip_code_macro', 'cip_code_micro',
       'cip_code_raw', 'title_code', 'series', 'cto', 'cto_code', 'prev_rank',
       'prev_gross_pay'],
      dtype='object')

In [21]:
len(df), len(all_pub)

(5214, 2091680)

In [22]:
df[df['uid']==9][['university', 'uid', 'yr', 'first_name', 'last_name', 'department',
       'field', 'title']]

,university,uid,yr,first_name,last_name,department,field,title


In [23]:
len(df), len(df.drop_duplicates())

(5214, 5214)

In [24]:
#make publication data on the (author_id, year) level

all_pub.groupby(['author_id'])

In [25]:
#merge publication and salary data
data_merge = all_pub.merge(df, on = ['university','first_name','last_name','yr'], how = 'left')
print(len(uc_pub_c))

NameError: name 'uc_pub_c' is not defined